<a href="https://colab.research.google.com/github/manuelboi/MLsec_project/blob/main/MLsec_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Models:
1. Fixing Data Augmentation to Improve Adversarial Robustness
2. Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?
3. MMA Training: Direct Input Space Margin Maximization through Adversarial Training

# General imports

In [1]:
%%capture

# Various
import matplotlib.pyplot as plt
import time
import importlib.util
import re

# Pytorch
import torch
from torch.utils.data import DataLoader, Subset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

# RobustBench
!pip install git+https://github.com/manuelboi/robustbench
from robustbench.utils import load_model
from robustbench.eval import benchmark
from robustbench.data import load_cifar10
from robustbench.model_zoo.enums import ThreatModel

# Smoothing
!git clone https://github.com/matteoturnu/smoothing.git
!conda create -n smoothing
!conda activate smoothing
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
!conda install scipy pandas statsmodels matplotlib seaborn
!pip install setGPU
from smoothing.code.core import Smooth
from smoothing.code.train import train, test
from smoothing.code.train_utils import AverageMeter, accuracy

# Setup

In [2]:
%%capture

# Set labels
labels_dct = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}

# Use GPU if available, otherwise use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Computing device used: ", device)

# Preparing trainset and testset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Set trainset and testset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define the number of samples you want to take from the whole dataset
num_samples = 1000

# Create a subset of the dataset with the chosen number of samples
indices = list(range(num_samples))
train_subset = Subset(trainset, indices)
test_subset = Subset(testset, indices)

# Preparing trainloader and testloader
batch_size = 64
trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=2)

# Load test samples for predict and certify
n_test_samples_benchmark = 100
x_test, y_test = load_cifar10(n_test_samples_benchmark)
x_test, y_test = x_test.to(device), y_test.to(device)

# Setting various parameters
epochs = 3 # used for training
sigma = 0.5 # gaussian noise standard deviation
n_examples = 20 # to be perturbed by AutoAttack
eps_L2 = 0.5 # epsilon of the perturbancy for L2 norm
eps_Linf = 8/255 # epsilon of the perturbancy with Linf norm
version = 'custom'
attacks_to_run=['apgd-ce', 'apgd-dlr'] # Attacks to run on the models

# Functions

Function to compact donwload and configuration of the model

In [3]:
def load_configure_model(name, dataset, threat_model, device):
  # Download model from robustbench
  model = load_model(model_name=name, dataset=dataset, threat_model=threat_model)

  # Set model to the chosen device
  model = model.to(device)

  # Set model in eval mode
  model.eval()

  return model

Function for smooth model training

In [4]:
def train_model(trainloader, testloader, model, epochs, sigma, device):
  criterion = CrossEntropyLoss().to(device)
  optimizer = SGD(model.parameters())
  scheduler = StepLR(optimizer, step_size=30)

  for epoch in range(epochs):
    scheduler.step(epoch)
    before = time.time()
    train_loss, train_acc = train(trainloader, model, criterion, optimizer, epoch, sigma, device)
    test_loss, test_acc = test(testloader, model, criterion, epoch, sigma, device)
    after = time.time()

  return model

Function for measuring performances on smooth model

In [21]:
def certify(model, sigma, x_text, y_test, L):
  n_classes = 10
  alpha = 0.5 # (1 - alpha) is the confidence level (in this case is 50%)
  n0 = 10 # number of samples for selection
  n = 20 # number of samples for estimation (certify) (too few samples but computation time is strongly affected with more)

  smooth_model = Smooth(model, n_classes, sigma)

  top_classes = list()
  radiuses = list()
  for x, y in zip(x_test, y_test):
    top_class = smooth_model.predict(x, n0, alpha, batch_size=n0, device=device,)
    top_class, radius = smooth_model.certify(x, n0, n, alpha, batch_size=n0, device=device, L=L)
    top_classes.append(top_class)
    radiuses.append(radius)

  top_classes = torch.tensor(top_classes, dtype=torch.float64).to(device)
  accuracy = torch.mean(top_classes == y_test, dtype=torch.float64)
  print("Radiuses list: ", radiuses)
  radius = torch.mean(torch.Tensor(radiuses), dtype=torch.float64)


  print("Top classes: ", top_classes)
  print("Y classes: ", y_test)
  print(f"Average radious found by certify: {float(radius.item())}")
  percentage = float(accuracy.item()) * 100
  print(f"Accuracy found by certify function: {percentage:.2f}%")

Function to compact test of the model with certification and AutoAttack

In [12]:
def test_model(model, sigma, x_test, y_test, norm, threat_model, n_examples, eps, batch_size, device, version, attacks_to_run):
  # Model prediction and certification on smoothed samples
  certify(model, sigma, x_test, y_test, norm)

  # AutoAttack on given model
  benchmark(model,
            threat_model=threat_model,
            n_examples=n_examples,
            eps=eps,
            batch_size=batch_size,
            device=device,
            version=version,
            attacks_to_run=attacks_to_run)

# Fixing Data Augmentation to Improve Adversarial Robustness (WideResNet-70-16)

L2

In [13]:
# Model loading
model_1_L2 = load_configure_model(name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_1_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_1_L2 = train_model(trainloader, testloader, model_1_L2, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_1_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

/usr/local/lib/python3.10/dist-packages/robustbench/utils.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=torch.device

Top classes:  tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3.], device='cuda:0',
       dtype=torch.float64)
Y classes:  tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7], device='cuda:0')
Average radious found by certify: inf
Accuracy found by certify function: 10.00%
Files already downloaded and verified
Cle


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/autoattack/checks.py", line 100, in check_dynamic
    sys.settrace(tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/autoattack/checks.py", line 102, in check_dynamic
    sys.settrace(None)



initial accuracy: 100.00%
apgd-ce - 1/1 - 1 out of 20 successfully perturbed
robust accuracy after APGD-CE: 95.00% (total time 287.8 s)
apgd-dlr - 1/1 - 1 out of 19 successfully perturbed
robust accuracy after APGD-DLR: 90.00% (total time 571.9 s)
max L2 perturbation: 0.50000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 90.00%
Adversarial accuracy: 90.00%


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRE

Epoch: [0][0/16]	Time 2.158 (2.158)	Data 0.199 (0.199)	Loss 0.6067 (0.6067)	Acc@1 82.812 (82.812)	Acc@5 100.000 (100.000)
Epoch: [0][1/16]	Time 0.610 (1.384)	Data 0.014 (0.106)	Loss 0.5613 (0.5840)	Acc@1 84.375 (83.594)	Acc@5 96.875 (98.438)
Epoch: [0][2/16]	Time 1.775 (1.515)	Data 0.004 (0.072)	Loss 0.4305 (0.5329)	Acc@1 87.500 (84.896)	Acc@5 100.000 (98.958)
Epoch: [0][3/16]	Time 1.753 (1.574)	Data 0.002 (0.055)	Loss 0.4194 (0.5045)	Acc@1 95.312 (87.500)	Acc@5 98.438 (98.828)
Epoch: [0][4/16]	Time 1.758 (1.611)	Data 0.003 (0.044)	Loss 0.4167 (0.4869)	Acc@1 89.062 (87.812)	Acc@5 100.000 (99.062)
Epoch: [0][5/16]	Time 1.757 (1.635)	Data 0.003 (0.037)	Loss 0.5305 (0.4942)	Acc@1 84.375 (87.240)	Acc@5 100.000 (99.219)
Epoch: [0][6/16]	Time 1.762 (1.653)	Data 0.003 (0.032)	Loss 0.3765 (0.4774)	Acc@1 92.188 (87.946)	Acc@5 100.000 (99.330)
Epoch: [0][7/16]	Time 1.759 (1.667)	Data 0.003 (0.029)	Loss 0.4343 (0.4720)	Acc@1 95.312 (88.867)	Acc@5 100.000 (99.414)
Epoch: [0][8/16]	Time 1.761 (1.67

Linf

In [14]:
# Model loading
model_1_Linf = load_configure_model(name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf', device=device)

# Test on stock model
test_model(model_1_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_1_Linf = train_model(trainloader, testloader, model_1_Linf, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_1_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

KeyboardInterrupt: 

# Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness? (ResNet-18)

L2

In [22]:
# Model loading
# WideResNet
#model_2_L2 = load_configure_model(name='Sehwag2021Proxy', dataset='cifar10', threat_model='L2', device=device)
# ResNet18
model_2_L2 = load_configure_model(name='Sehwag2021Proxy_R18', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_2_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_2_L2 = train_model(trainloader, testloader, model_2_L2, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_2_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

Radiuses list:  [0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.4566355681940677, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.6941933567387186, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.9120818020291023, 0.6941933567387186, 0.9120818020291023, 0.9120818020291023, 0.6941933567387186, 0.9120818020291023, 0.9

Linf

In [16]:
# Model loading
#model_2_Linf = load_configure_model(name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf', device=device)
# ResNet18
model_2_Linf = load_configure_model(name='Sehwag2021Proxy_R18', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_2_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_2_Linf = train_model(trainloader, testloader, model_2_Linf, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_2_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

NameError: name 'model_2_Linf' is not defined

# MMA Training: Direct Input Space Margin Maximization through Adversarial Training (WideResNet-28-4)

L2

In [ ]:
# Model loading
model_3_L2 = load_configure_model(name='Ding2020MMA', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_3_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_3_L2 = train_model(trainloader, testloader, model_3_L2, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_3_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

Linf

In [ ]:
# Model loading
model_3_Linf = load_configure_model(name='Ding2020MMA', dataset='cifar10', threat_model='Linf', device=device)

# Test on stock model
test_model(model_3_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_3_Linf = train_model(trainloader, testloader, model_3_Linf, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_3_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)